# Auto-ablation with Howso Engine

> Note: This feature is experimental and is not universally recommended for production deployment.

## Overview

This notebook provides an overview of using auto-ablation during train to reduce the size of your data _as they are trained_.

In [1]:
import pandas as pd
from pmlb import fetch_data

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Step 1: Load Data & Feature Mapping

Our example dataset for this recipe is the well-known `Adult` dataset. This dataset works well because it has over 48,000 cases. In general, datasets containing over 25,000 rows work well. The default minimum size is 1,000 cases, below which no cases will be ablated.

This part of the process is identical to other recipes.

In [2]:
df = fetch_data("adult", local_cache_dir="../../../data")
features = infer_feature_attributes(df)

display(df)
display(features.to_dataframe())

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39.0,7,77516.0,9,13.0,4,1,1,4,1,2174.0,0.0,40.0,39,1
1,50.0,6,83311.0,9,13.0,2,4,0,4,1,0.0,0.0,13.0,39,1
2,38.0,4,215646.0,11,9.0,0,6,1,4,1,0.0,0.0,40.0,39,1
3,53.0,4,234721.0,1,7.0,2,6,0,2,1,0.0,0.0,40.0,39,1
4,28.0,4,338409.0,9,13.0,2,10,5,2,0,0.0,0.0,40.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39.0,4,215419.0,9,13.0,0,10,1,4,0,0.0,0.0,36.0,39,1
48838,64.0,0,321403.0,11,9.0,6,0,2,2,1,0.0,0.0,40.0,39,1
48839,38.0,4,374983.0,9,13.0,2,10,0,4,1,0.0,0.0,50.0,39,1
48840,44.0,4,83891.0,9,13.0,0,1,3,1,1,5455.0,0.0,40.0,39,1


type decimal_places bounds                        \
                                             min        max allow_null   
age             continuous              0    0.0      137.0       True   
workclass          nominal              0    NaN        NaN      False   
fnlwgt          continuous              0    0.0  2449285.0       True   
education          nominal              0    NaN        NaN      False   
education-num   continuous              0    0.0       26.0       True   
marital-status     nominal              0    NaN        NaN      False   
occupation         nominal              0    NaN        NaN      False   
relationship       nominal              0    NaN        NaN      False   
race               nominal              0    NaN        NaN      False   
sex                nominal              0    NaN        NaN      False   
capital-gain    continuous              0    0.0   164870.0       True   
capital-loss    continuous              0    0.0     7182.0       True   
hours-per-week  continuous              0    0.0      163.0       True   
native-country     nominal              0    NaN        NaN      False   
target             nominal              0    NaN        NaN      False   

                                         data_type original_type       
               observed_min observed_max               data_type size  
age                    17.0         90.0    number       numeric    8  
workclass               NaN          NaN    number       integer    8  
fnlwgt              12285.0    1490400.0    number       numeric    8  
education               NaN          NaN    number       integer    8  
education-num           1.0         16.0    number       numeric    8  
marital-status          NaN          NaN    number       integer    8  
occupation              NaN          NaN    number       integer    8  
relationship            NaN          NaN    number       integer    8  
race                    NaN          NaN    number       integer    8  
sex                     NaN          NaN    number       integer    8  
capital-gain            0.0      99999.0    number       numeric    8  
capital-loss            0.0       4356.0    number       numeric    8  
hours-per-week          1.0         99.0    number       numeric    8  
native-country          NaN          NaN    number       integer    8  
target                  NaN          NaN    number       integer    8

## Step 3: Create Trainee and Set Parameters

The process of creating the `Trainee` is the same as the other recipes. However, after the `Trainee` is created, we set auto-analyze and auto-ablation parameters to ensure that the `Trainee` remains informed as it removes cases during train.

In [3]:
t = Trainee(features=features)

# Set auto-analyze parameters. Note that use_case_weights=True since case-weights are how
# information from ablated cases is retained in the Trainee.
t.set_auto_analyze_params(auto_analyze_enabled=True, use_case_weights=True)

# Set auto-ablation parameters.
t.set_auto_ablation_params(
    auto_ablation_enabled=True,
    influence_weight_entropy_threshold=0.6,
    min_num_cases=2_500,
    max_num_cases=10_000,
    delta_threshold_map={
        "accuracy": {"target": 0.1}
    }
)

This recipe uses the following parameters for auto-ablation:

- `auto_ablation_enabled` — When this parameter is `False`, auto-ablation will not be performed.
- `influence_weight_entropy_threshold` — This parameter is used to determine which cases will be ablated.
  If the entropy of a case's influence weights would be above this percentile of existing cases' influence
  weight entropies, then the case is ablated. This is set to $0.6$, the default value.
- `min_num_cases` — This parameter sets the minimum number of cases that are allowed to be in the Trainee.
- `max_num_cases` — This parameter sets the maximum number of cases that are allowed to be in the Trainee.
- `delta_threshold_map` — This parameter defines thresholds that are used to determine when ablation should stop.
  In this case, the threshold map instructs auto-ablation to stop if the delta between the current accuracy
  of the `"target"` feature and its previous accuracy is greater than $0.1$. I.e., ablation will stop if accuracy
  for that feature drops from $0.8$ to $0.7$. More formally, ablation will stop if
  $\text{acc}_\text{old} - \text{acc}_\text{new} \geq 0.1$.

## Step 4: Training

Now that the parameters are set, the `Trainee` can be trained as if this were a normal workflow.

In [4]:
t.train(df)

Notably, only a subset of the cases have been trained (the rest have been ablated).

In [5]:
num_trained_cases = t.get_num_training_cases()
num_total_cases = len(df)
num_trained_percent = round((num_trained_cases / num_total_cases) * 100, 3)

print(f"Trained {num_trained_cases:,} cases out of {num_total_cases:,} ({num_trained_percent}%).")

Trained 3,120 cases out of 48,842 (6.388%).


## Step 5: Results

An ablated `Trainee` can be used in much the same way as other `Trainee`s, as long as `use_case_weights=True`. Here, we will investigate the prediction stats for all features.

In [8]:
stats = t.react_aggregate(
    prediction_stats_action_feature="target",
    details={
        "prediction_stats": True,
        "selected_prediction_stats": ["all"],
    },
    use_case_weights=True,
)

stats_df = stats.to_dataframe()
stats_df

,target,education,marital-status,workclass,sex,relationship,native-country,occupation,race,hours-per-week,capital-gain,capital-loss,fnlwgt,education-num,age
accuracy,0.870000,1.0,0.864000,0.932000,0.842000,0.846000,0.974773,0.377000,0.959960,NaN,NaN,NaN,NaN,NaN,NaN
adjusted_smape,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.495975,81.010623,139.054028,46.715468,6.907632,22.856769
mae,0.136835,0.0,0.152524,0.102813,0.177977,0.181856,0.047870,0.630129,0.061449,7.700870,2249.376355,250.308889,81986.625446,0.617156,8.967695
mcc,0.601221,1.0,0.781368,0.700185,0.545397,0.762684,0.458544,0.297153,0.033737,NaN,NaN,NaN,NaN,NaN,NaN
missing_value_accuracy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
precision,0.802795,1.0,0.459406,0.420019,0.793093,0.483323,0.287523,0.281906,0.349509,NaN,NaN,NaN,NaN,NaN,NaN
r2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.183947,-0.257199,-0.375391,-0.440745,0.775711,0.294505
recall,0.798442,1.0,0.456584,0.374374,0.753723,0.448576,0.298137,0.273497,0.342030,NaN,NaN,NaN,NaN,NaN,NaN
rmse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.198171,9241.326815,520.333601,108244.468469,1.283219,11.876140
smape,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.823151,82.194327,141.913405,46.715619,7.386641,23.174343


In [9]:
# The confusion matrix can be retrieved
print("Howso Prediction Results - Confusion Matrix for 'target'")
matrix = pd.DataFrame(stats['confusion_matrix']['target']["matrix"])
matrix.index.name = "Predicted"
matrix.columns.name = "Actual"
display(matrix)

Howso Prediction Results - Confusion Matrix for 'target'


Actual,0,1
Predicted,,
0,140,63
1,67,730


## Conclusion

Howso Engine with auto-ablation can compress data to a fraction of its original size while ensuring accuracy (and other measures) remain in an acceptable range without compromising the other capabilities of the `Trainee`.